### Homework 8
**1) Data Fitting vs. Sparsity Tradeof**

In [1]:
import scipy.io as sio
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
rand = 1100

In [2]:
mat_contents = sio.loadmat('BreastCancer.mat')
X = mat_contents['X']
y = mat_contents['y']

train = X[:100, :]
train_labels = y[:100, :]

test = X[100:, :]
test_labels = y[100:, :]

In [128]:
X = np.random.rand(10,1000)
y = np.random.rand(10,1000)

In [ ]:
regr = ElasticNet(random_state=0)
regr.fit(X, y)
# print(regr.coef_)
print(np.linalg.norm(y - X@regr.coef_))

coef = lasso(X,y, 1)
print(np.linalg.norm(y - X@coef))

coef = ridge(X,y, 1)
print(np.linalg.norm(y - X@coef))

57.97862777541715


In [10]:
def ridge(A, b, gamma):
    num_rows, num_cols = A.shape
    
     # (A^T*A + gamma*I)^-1 * A^T * b
    g  = np.zeros((num_cols, num_cols))
    np.fill_diagonal(g, gamma)
    return np.linalg.inv(A.T @ A + g) @ A.T @ b

In [6]:
norm = lambda x: np.linalg.norm(x)
def lasso(A, b, gamma):
    max_iter = int(1e6)
    tol = 1e-5
    
    tau = 1/norm(A)**2
    
    _, cols = A.shape
    
    w = np.zeros((cols, 1))
    
    for i in range(1, max_iter):
        z = w - tau*(A.T @ (A @ w - b))
        w_old = w
        w = np.sign(z)*np.maximum(np.absolute(z) - tau*gamma/2, 0)
        if norm(w - w_old) < tol:
            break
    return w

In [35]:
lasso(X, y, .1)

array([[0.        ],
       [0.49910629]])

#### Initial Training

In [ ]:
gamma_test = [100, 10, 1, .05, .00001]
results = []

l2_norm = lambda x: np.linalg.norm(x, 2)
for g in gamma_test:
    results.append(lasso(train, train_labels, g))
    print(g)

In [ ]:
resid = []
w_norm = []
l1_norm = lambda x: np.linalg.norm(x, ord=1)
results = np.asarray(results)
for r in results:
    y_approx = train @ r
    w_norm.append(np.linalg.norm((r), ord=1))
    resid.append(l2_norm(train_labels - y_approx))

#### Training Plot

In [ ]:
plt.scatter(resid, w_norm)

#### Validation Training

In [ ]:
gamma_test = [100, 10, 1, .05, .00001]
results = []

l2_norm = lambda x: np.linalg.norm(x, 2)
for g in gamma_test:
    results.append(lasso(test, test_labels, g))
    print(g)

In [ ]:
results[0].shape

In [ ]:
resid = []
w_norm = []
l1_norm = lambda x: np.linalg.norm(x, ord=1)
results = np.asarray(results)
for r in results:
    y_approx = test @ r
    w_norm.append(np.linalg.norm((r), ord=1))
    resid.append(l2_norm(test_labels - y_approx))

#### Validation Plot

In [ ]:
plt.scatter(resid, w_norm)

#### Lasso vs Ridge

In [ ]:
def ridge(A, b, gamma):
    _, num_cols = A.shape
    
     # (A^T*A + gamma*I)^-1 * A^T * b
    g  = np.zeros((num_cols, num_cols))
    np.fill_diagonal(g, gamma)
    # return np.linalg.inv(A.transpose().dot(A) + g).dot(A.transpose()).dot(b)
    return np.linalg.inv(A.T @ A + g)@ A.T @ b

#### Cross Validation

In [ ]:
def ridge_error(w, test, y):
    return np.linalg.norm(y - test@w) 

def cross_validation(X_test, y_test):
    error_lasso = []
    error_ridge = []
    
    ridge_ws = []
    lasso_ws = []

    gamma = [.05, .001, .005, .00001, .00005, 1e-5, 5e-5, 1e-6]
    
    for i, g in enumerate(gamma):
        if i <= 5:
            X_1, X_test, y_1, y_test = train_test_split(X_test, y_test, train_size = 30, random_state=rand)
        else:
            X_1, X_test, y_1, y_test = train_test_split(X_test, y_test, train_size = 29, random_state=rand)
        
        ridge_w = np.asarray(ridge(X_1, y_1, g))
        lasso_w = np.asarray(lasso(X_1, y_1, g))
        
        ridge_ws.append(ridge_w)
        lasso_ws.append(lasso_w)
        print(i)
     
    X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, train_size = 29, random_state=rand) 
    
    best_ridge_error = ridge_error(ridge_ws[0], X_valid, y_valid)
    best_lasso_error = ridge_error(lasso_ws[0], X_valid, y_valid)
    best_ridge_w = ridge_ws[0]
    best_lasso_w = lasso_ws[0]
    for i in range(1, len(gamma)):
        ridge_error_1 = ridge_error(ridge_ws[i], X_valid, y_valid)
        lasso_error_1 = ridge_error(lasso_ws[i], X_valid, y_valid)
        
        if ridge_error_1 < best_ridge_error:
            best_ridge_w = ridge_ws[i]
            best_ridge_error = ridge_error_1
        
        if lasso_error_1 < best_lasso_error:
            best_lasso_w = lasso_ws[i]
            best_lasso_error = lasso_error_1
            
    return best_lasso_w, best_lasso_error, best_ridge_w, best_ridge_error, X_test, y_test

In [ ]:
lasso_w, lasso_error, ridge_w, ridge_error, X_test, y_test = cross_validation(X, y)

In [ ]:
def classification_error(w, test, y):
    y_hat = test@w
    signs = np.sign(y_hat)
    correct_classifcation_percentage = np.sum(np.equal(signs, y))/len(y)
    residual_error = resid_error(w, test, y)
    return correct_classifcation_percentage, residual_error

def resid_error(w, test, y):
    return np.linalg.norm(y - test@w) 

In [ ]:
lasso_classification_error, lasso_residual = classification_error(lasso_w, X_test, y_test)
ridge_classification_error, ridge_residual = classification_error(ridge_w, X_test, y_test)

In [ ]:
print("Lasso classification error ", lasso_classification_error, " Ridge classification Error ", ridge_classification_error)
print("Lasso residual ", lasso_classification_error, " Ridge residual ", ridge_classification_error)